# Utilisation des réseaux euclidiens dans pour le décodage de code de dimension 1

## Génération d'équations


On va essayer de décoder des équations de la forme : 

\begin{equation}
 y_i +C_ix + D_i \equiv 0 [p]
\end{equation}
    Que l'on peut reformuler en :
\begin{equation}
y_i + A_i y_0 + B_i \equiv 0 [p] , i = 1\dots n
\end{equation}

avec $y_0=x$ ou $y_0=y_h$

On va d'abord générer les différents $y_i$

In [2]:
prime_size = 160
h = 2
y = []
p = random_prime(2^prime_size-1, lbound=2^(prime_size-1))
F = GF(p)
for i in range(h):
    y.append(F(randint(0,p-1)))
    
len(y)

2

On va maintenant générer $h-1$ équations :

In [3]:
a = []
b = []
v = vector(ZZ,h)
v[0] = 0
sample = []
arranged_sample = []
known_bytes = 160
mask = 0
for i in range(known_bytes):
    mask += 1<<i
z = vector(ZZ,h)
c0 = y[h-1].lift()&mask
z[0] = y[h-1].lift()-c0

for i in range(h-1):
    j = i+1
    a.append(F(randint(0,p-1)))
    b.append(F(-(y[i]+a[i]*y[h-1])))
    sample.append(y[i].lift()+(a[i]*y[h-1]).lift() + b[i].lift()) 
    ci = y[i].lift()&mask
    zi = y[i].lift()-ci
    z[j] = zi
    v[j] = ci + a[i]*c0+b[i]
    arranged_sample.append(z[j] + a[i]*z[0]+ v[j])
    print(sample[i]==arranged_sample[i])

True


In [4]:
A = zero_matrix(ZZ,h)
A[0,0] = -1
for k in range(1,h):
    A[0,k] = a[k-1]
    A[k,k] = p

def Babai_LLL(A, target):
    B = A.LLL()
    G = B.gram_schmidt()[0]
    diff = target
    for i in reversed(range(B.nrows())):
        diff -=  B[i] * ((diff * G[i]) / (G[i] * G[i])).round()
    return target - diff
print(A.str()+ "\n" + str(a) + "\n" + str(p))

[                                              -1 412653007807629137806006525389276667250529272194]
[                                               0 901617521109299542381301077865357135693349829103]
[412653007807629137806006525389276667250529272194]
901617521109299542381301077865357135693349829103


In [5]:
w = Babai_LLL(A,v)
print(str((w-v)) + " , " + str(z) + "," + str(y))

(0, 0) , (0, 0),[409440745122097032809139815645505058958313728341, 22492187799397341182829709622749166551868515476]


In [6]:
print(str(w) + "," + str(v))

(0, 0),(0, 0)


In [314]:
z+v

(0, 587)